# Weni Project Cloner

This notebook copies information from one Weni project to another using the Weni API.

## Features:
1. Copy agent customization (name, role, personality, goal, instructions)
2. Copy knowledge bases:
   - Text bases
   - Site/link bases
   - File bases

## 1. Setup and Configuration

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
import time
from typing import Dict, List, Any

# Load environment variables
load_dotenv()

# Get configuration from .env
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
SOURCE_PROJECT_UUID = os.getenv('SOURCE_PROJECT_UUID')
DESTINATION_PROJECT_UUID = os.getenv('DESTINATION_PROJECT_UUID')

# Validate configuration
if not all([BEARER_TOKEN, SOURCE_PROJECT_UUID, DESTINATION_PROJECT_UUID]):
    raise ValueError("Please ensure all required environment variables are set in .env file")

print(f"Source Project: {SOURCE_PROJECT_UUID}")
print(f"Destination Project: {DESTINATION_PROJECT_UUID}")
print("Configuration loaded successfully!")

Source Project: 44e71074-cc9d-40f1-9640-546572f842c8
Destination Project: 45718786-4066-4338-9e86-f3ea525224d2
Configuration loaded successfully!


## 2. API Helper Functions

In [2]:
# Common headers for all requests
def get_headers():
    return {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es;q=0.6,nl;q=0.5,fr;q=0.4',
        'authorization': f'Bearer {BEARER_TOKEN}',
        'origin': 'https://intelligence-next.weni.ai',
        'priority': 'u=1, i',
        'referer': 'https://intelligence-next.weni.ai/',
        'sec-ch-ua': '"Not;A=Brand";v="99", "Google Chrome";v="139", "Chromium";v="139"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36'
    }

def get_headers_with_content_type():
    headers = get_headers()
    headers['content-type'] = 'application/json'
    return headers

def get_content_base_uuid(project_uuid: str) -> str:
    """Get content base UUID from project UUID using router API"""
    url = f'https://nexus.weni.ai/api/{project_uuid}/router/'
    
    response = requests.get(url, headers=get_headers())
    
    if response.status_code == 200:
        router_data = response.json()
        content_base_uuid = router_data.get('uuid')
        if content_base_uuid:
            return content_base_uuid
        else:
            raise Exception(f"No UUID found in router response for project {project_uuid}")
    else:
        raise Exception(f"Failed to get router data: {response.status_code} - {response.text}")

## 3. Agent Customization Functions

In [3]:
def get_agent_customization(project_uuid: str) -> Dict[str, Any]:
    """Get agent customization from source project"""
    url = f'https://nexus.weni.ai/api/{project_uuid}/customization/'
    
    response = requests.get(url, headers=get_headers())
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to get agent customization: {response.status_code} - {response.text}")

def update_agent_customization(project_uuid: str, customization_data: Dict[str, Any]) -> Dict[str, Any]:
    """Update agent customization in destination project"""
    url = f'https://nexus.weni.ai/api/{project_uuid}/customization/'
    
    response = requests.put(url, headers=get_headers_with_content_type(), json=customization_data)
    
    if response.status_code in [200, 201]:
        return response.json()
    else:
        raise Exception(f"Failed to update agent customization: {response.status_code} - {response.text}")

## 4. Knowledge Base Functions

In [4]:
# Text Base Functions
def get_text_bases(content_base_uuid: str) -> List[Dict[str, Any]]:
    """Get all text bases from content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-text/'
    
    response = requests.get(url, headers=get_headers())
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to get text bases: {response.status_code} - {response.text}")

def create_text_base(content_base_uuid: str, text_data: Dict[str, str]) -> Dict[str, Any]:
    """Create a text base in destination content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-text/'
    
    response = requests.post(url, headers=get_headers_with_content_type(), json=text_data)
    
    if response.status_code in [200, 201]:
        return response.json()
    else:
        raise Exception(f"Failed to create text base: {response.status_code} - {response.text}")

def update_text_base(content_base_uuid: str, text_base_uuid: str, text_data: Dict[str, str]) -> Dict[str, Any]:
    """Update an existing text base in destination content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-text/{text_base_uuid}/'
    
    response = requests.put(url, headers=get_headers_with_content_type(), json=text_data)
    
    if response.status_code in [200, 201]:
        return response.json()
    else:
        raise Exception(f"Failed to update text base: {response.status_code} - {response.text}")

In [5]:
# Test getting content base UUID and text bases
source_content_base_uuid = get_content_base_uuid(SOURCE_PROJECT_UUID)
destination_content_base_uuid = get_content_base_uuid(DESTINATION_PROJECT_UUID)

print(f"Source content base UUID: {source_content_base_uuid}")
print(f"Destination content base UUID: {destination_content_base_uuid}")

# Test source text bases
source_text_response = get_text_bases(source_content_base_uuid)
source_text_bases = source_text_response.get("results", [])
print(f"Found {len(source_text_bases)} text bases in source")

# Test destination text bases
destination_text_response = get_text_bases(destination_content_base_uuid)
destination_text_bases = destination_text_response.get("results", [])
print(f"Found {len(destination_text_bases)} text bases in destination")

print(f"First source text base: {source_text_bases[0] if source_text_bases else 'None'}")
print(f"First destination text base: {destination_text_bases[0] if destination_text_bases else 'None'}")

Source content base UUID: bedaa02d-b6fe-4a19-abf9-e2e4e87beaaa
Destination content base UUID: 3bea9443-4f5b-4736-97d7-dfd2285fe00a
Found 1 text bases in source
Found 3 text bases in destination
First source text base: {'text': 'Canais de atendimento:\n- Entre em contato com a Central do cliente: \nTelefone: 3003-3400 na opção 3\nSegunda a sexta: das 08h às 18h\nSábados: das 08h às 13h\nWhatsapp no link: https://wa.me/551130033400\nou pelo portal do cliente: https://portalcliente.obramax.com.br\n- Entre em contato com o Televendas pelo Telefone: 3003-3400 na opção 2\nSegunda a sexta: das 08h às 18h Sábados: das 08h às 13h\nou Faça seu orçamento no Orçamax acessando o link https://www.obramax.com.br/orcamax\n\nO cliente pode retirar o seu pedido nas nossas lojas físicas de acordo com a loja escolhida no checkout:\nNOSSAS LOJAS FÍSICAS:\n- ES - Cariacica:\nAv. Mário Gurgel, S/N- Oriente, Cariacica - ES, 29147-345\nSegunda a Sábado das 7h às 21h\nDomingos das 9h às 17h\nFeriados das 8h às 

In [6]:
# Link/Site Base Functions
def get_link_bases(content_base_uuid: str) -> List[Dict[str, Any]]:
    """Get all link bases from content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-link/'
    
    response = requests.get(url, headers=get_headers())
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to get link bases: {response.status_code} - {response.text}")

def create_link_base(content_base_uuid: str, link_data: Dict[str, str]) -> Dict[str, Any]:
    """Create a link base in destination content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-link/'
    
    response = requests.post(url, headers=get_headers_with_content_type(), json=link_data)
    
    if response.status_code in [200, 201]:
        return response.json()
    else:
        raise Exception(f"Failed to create link base: {response.status_code} - {response.text}")

In [7]:
# File Base Functions
def get_file_bases(content_base_uuid: str) -> List[Dict[str, Any]]:
    """Get all file bases from content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-file/'
    
    response = requests.get(url, headers=get_headers())
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to get file bases: {response.status_code} - {response.text}")

def download_file(file_url: str) -> bytes:
    """Download a file from URL"""
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception(f"Failed to download file: {response.status_code}")

def create_file_base(content_base_uuid: str, file_data: Dict[str, Any], file_content: bytes) -> Dict[str, Any]:
    """Create a file base in destination content base"""
    url = f'https://nexus.weni.ai/api/{content_base_uuid}/content-bases-file/'
    
    # Prepare multipart form data
    files = {
        'file': (file_data.get('filename', 'file.pdf'), file_content, file_data.get('content_type', 'application/pdf'))
    }
    
    data = {
        'extension_file': file_data.get('extension_file', 'pdf'),
        'load_type': file_data.get('load_type', 'pdfminer')
    }
    
    # Remove content-type header for multipart
    headers = get_headers()
    
    response = requests.post(url, headers=headers, files=files, data=data)
    
    if response.status_code in [200, 201]:
        return response.json()
    else:
        raise Exception(f"Failed to create file base: {response.status_code} - {response.text}")

## 5. Copy Agent Customization

In [8]:
def copy_agent_customization():
    """Copy agent customization from source to destination project"""
    print("Copying agent customization...")
    
    try:
        # Get source customization
        source_customization = get_agent_customization(SOURCE_PROJECT_UUID)
        print(f"Retrieved agent customization from source project")
        print(f"Agent name: {source_customization.get('agent', {}).get('name', 'N/A')}")
        print(f"Number of instructions: {len(source_customization.get('instructions', []))}")

        print(source_customization)
        
        # Prepare destination data - strip IDs from instructions
        destination_data = {
            "agent": source_customization.get("agent", {}),
            "instructions": [
                {"instruction": instr["instruction"]} 
                for instr in source_customization.get("instructions", [])
            ]
        }
        
        print(f"\nPrepared destination data (without IDs):")
        print(destination_data)
        
        # Update destination with processed data
        result = update_agent_customization(DESTINATION_PROJECT_UUID, destination_data)
        print("✅ Successfully copied agent customization to destination project")
        
        return result
        
    except Exception as e:
        print(f"❌ Error copying agent customization: {str(e)}")
        return None

In [9]:
# Execute agent customization copy
agent_result = copy_agent_customization()

Copying agent customization...
Retrieved agent customization from source project
Agent name: Vendedor Virtual
Number of instructions: 39
{'agent': {'name': 'Vendedor Virtual', 'role': 'Especialista em Obras', 'personality': 'Amigável', 'goal': 'Você tira dúvidas e oferta produtos de construção civil (banheiros, climatização e ventilação, cozinha e área de serviço, EPIs, ferragens, ferramentas, iluminação, impermeabilizantes, marcenaria e madeiras, materiais de construção, materiais elétricos, materiais hidráulicos, organização e limpeza, pisos e revestimentos, portas e janelas, sistemas de segurança e comunicação, e tintas e acessórios) no varejo ou atacado. Suas funcionalidades são: buscar produtos, gerar link de pagamento do checkout, fornecer informações sobre o pedido do cliente e nota fiscal. Seu público alvo são profissionais da construção como pedreiros, encanadores, eletricistas, engenheiros, mestres de obra e arquitetos'}, 'instructions': [{'id': 10004, 'instruction': 'Se não 

## 6. Copy Text Bases

In [10]:
def copy_text_bases():
    """Copy all text bases from source to destination project"""
    print("\nCopying text bases...")
    
    try:
        # Get content base UUIDs from project UUIDs
        print("Getting content base UUIDs...")
        source_content_base_uuid = get_content_base_uuid(SOURCE_PROJECT_UUID)
        destination_content_base_uuid = get_content_base_uuid(DESTINATION_PROJECT_UUID)
        print(f"Source content base UUID: {source_content_base_uuid}")
        print(f"Destination content base UUID: {destination_content_base_uuid}")
        
        # Get source text bases
        source_response = get_text_bases(source_content_base_uuid)
        source_text_bases = source_response.get("results", [])
        print(f"Found {len(source_text_bases)} text bases in source project")
        
        # Get existing destination text bases
        destination_response = get_text_bases(destination_content_base_uuid)
        destination_text_bases = destination_response.get("results", [])
        print(f"Found {len(destination_text_bases)} existing text bases in destination project")
        
        copied_count = 0
        
        for i, source_text_base in enumerate(source_text_bases):
            try:
                # Create text base data
                text_data = {
                    "text": source_text_base.get("text", "")
                }
                
                # Check if we should update an existing text base or create a new one
                if i < len(destination_text_bases):
                    # Update existing text base
                    dest_text_base_uuid = destination_text_bases[i]["uuid"]
                    result = update_text_base(destination_content_base_uuid, dest_text_base_uuid, text_data)
                    print(f"  ✅ Updated text base {i+1}/{len(source_text_bases)}: {text_data['text'][:50]}...")
                else:
                    # Create new text base
                    result = create_text_base(destination_content_base_uuid, text_data)
                    print(f"  ✅ Created text base {i+1}/{len(source_text_bases)}: {text_data['text'][:50]}...")
                
                copied_count += 1
                
                # Small delay to avoid rate limiting
                time.sleep(0.5)
                
            except Exception as e:
                print(f"  ❌ Error copying text base {i+1}: {str(e)}")
        
        print(f"\n✅ Successfully copied {copied_count}/{len(source_text_bases)} text bases")
        
    except Exception as e:
        print(f"❌ Error accessing text bases: {str(e)}")

In [11]:
# Execute text bases copy
copy_text_bases()


Copying text bases...
Getting content base UUIDs...
Source content base UUID: bedaa02d-b6fe-4a19-abf9-e2e4e87beaaa
Destination content base UUID: 3bea9443-4f5b-4736-97d7-dfd2285fe00a
Found 1 text bases in source project
Found 3 existing text bases in destination project
  ✅ Updated text base 1/1: Canais de atendimento:
- Entre em contato com a Ce...

✅ Successfully copied 1/1 text bases


## 7. Copy Link/Site Bases

In [12]:
def copy_link_bases():
    """Copy all link bases from source to destination project"""
    print("\nCopying link/site bases...")
    
    try:
        # Get content base UUIDs from project UUIDs
        print("Getting content base UUIDs...")
        source_content_base_uuid = get_content_base_uuid(SOURCE_PROJECT_UUID)
        destination_content_base_uuid = get_content_base_uuid(DESTINATION_PROJECT_UUID)
        print(f"Source content base UUID: {source_content_base_uuid}")
        print(f"Destination content base UUID: {destination_content_base_uuid}")
        
        # Get source link bases
        source_link_bases = get_link_bases(source_content_base_uuid)
        # Link bases return a direct array, not wrapped in results
        print(f"Found {len(source_link_bases)} link bases in source project")
        
        copied_count = 0
        
        for i, link_base in enumerate(source_link_bases):
            try:
                # Create link base data
                link_data = {
                    "link": link_base.get("link", "")
                }
                
                # Create in destination
                result = create_link_base(destination_content_base_uuid, link_data)
                copied_count += 1
                print(f"  ✅ Copied link base {i+1}/{len(source_link_bases)}: {link_data['link']}")
                
                # Small delay to avoid rate limiting
                time.sleep(0.5)
                
            except Exception as e:
                print(f"  ❌ Error copying link base {i+1}: {str(e)}")
        
        print(f"\n✅ Successfully copied {copied_count}/{len(source_link_bases)} link bases")
        
    except Exception as e:
        print(f"❌ Error accessing link bases: {str(e)}")

In [13]:
# Execute link bases copy
copy_link_bases()


Copying link/site bases...
Getting content base UUIDs...
Source content base UUID: bedaa02d-b6fe-4a19-abf9-e2e4e87beaaa
Destination content base UUID: 3bea9443-4f5b-4736-97d7-dfd2285fe00a
Found 6 link bases in source project
  ✅ Copied link base 1/6: https://blog.obramax.com.br/
  ✅ Copied link base 2/6: https://www.obramax.com.br/perguntas-frequentes
  ✅ Copied link base 3/6: https://www.obramax.com.br/retira-facil
  ✅ Copied link base 4/6: https://www.obramax.com.br/politica-de-troca-e-devolucao
  ✅ Copied link base 5/6: https://www.obramax.com.br/politica-de-entrega
  ✅ Copied link base 6/6: https://www.obramax.com.br/sobre-obramax

✅ Successfully copied 6/6 link bases


## 8. Copy File Bases

In [14]:
def copy_file_bases():
    """Copy all file bases from source to destination project"""
    print("\nCopying file bases...")
    
    try:
        # Get content base UUIDs from project UUIDs
        print("Getting content base UUIDs...")
        source_content_base_uuid = get_content_base_uuid(SOURCE_PROJECT_UUID)
        destination_content_base_uuid = get_content_base_uuid(DESTINATION_PROJECT_UUID)
        print(f"Source content base UUID: {source_content_base_uuid}")
        print(f"Destination content base UUID: {destination_content_base_uuid}")
        
        # Get source file bases
        source_response = get_file_bases(source_content_base_uuid)
        source_file_bases = source_response.get("results", [])
        print(f"Found {len(source_file_bases)} file bases in source project")
        
        copied_count = 0
        
        for i, file_base in enumerate(source_file_bases):
            try:
                # Download the file - use file_name field from response
                file_url = file_base.get('file')
                file_name = file_base.get('file_name', f'file_{i+1}.pdf')
                
                if not file_url:
                    print(f"  ⚠️  No file URL for file base {i+1} ({file_name}), skipping")
                    continue
                
                print(f"  Downloading file {i+1}/{len(source_file_bases)}: {file_name}...")
                file_content = download_file(file_url)
                
                # Prepare file data using correct field names from API response
                file_data = {
                    'filename': file_base.get('created_file_name', file_name.split('.')[0]),
                    'extension_file': file_base.get('extension_file', 'pdf'),
                    'load_type': 'pdfminer',  # Default load type
                    'content_type': 'application/pdf'  # Default content type
                }
                
                # Create in destination
                result = create_file_base(destination_content_base_uuid, file_data, file_content)
                copied_count += 1
                print(f"  ✅ Copied file base {i+1}/{len(source_file_bases)}: {file_name}")
                
                # Small delay to avoid rate limiting
                time.sleep(1)
                
            except Exception as e:
                print(f"  ❌ Error copying file base {i+1}: {str(e)}")
        
        print(f"\n✅ Successfully copied {copied_count}/{len(source_file_bases)} file bases")
        
    except Exception as e:
        print(f"❌ Error accessing file bases: {str(e)}")

In [15]:
# Execute file bases copy
copy_file_bases()


Copying file bases...
Getting content base UUIDs...
Source content base UUID: bedaa02d-b6fe-4a19-abf9-e2e4e87beaaa
Destination content base UUID: 3bea9443-4f5b-4736-97d7-dfd2285fe00a
Found 0 file bases in source project

✅ Successfully copied 0/0 file bases


## 9. Complete Copy Process

In [ ]:
def copy_all_project_data():
    """Copy all data from source to destination project"""
    print("=" * 50)
    print("Starting complete project copy...")
    print(f"Source: {SOURCE_PROJECT_UUID}")
    print(f"Destination: {DESTINATION_PROJECT_UUID}")
    print("=" * 50)
    
    # Copy agent customization
    copy_agent_customization()
    
    # Copy text bases
    copy_text_bases()
    
    # Copy link bases
    copy_link_bases()
    
    # Copy file bases
    copy_file_bases()
    
    print("\n" + "=" * 50)
    print("✅ Project copy process completed!")
    print("=" * 50)

In [ ]:
# Execute complete copy process
copy_all_project_data()

## 10. Verification Functions (Optional)

In [ ]:
def verify_copy():
    """Verify that all data was copied correctly"""
    print("\nVerifying copy results...")
    print("=" * 50)
    
    try:
        # Get destination content base UUID
        destination_content_base_uuid = get_content_base_uuid(DESTINATION_PROJECT_UUID)
        
        # Check agent customization
        dest_customization = get_agent_customization(DESTINATION_PROJECT_UUID)
        print(f"✅ Agent customization: {dest_customization.get('agent', {}).get('name', 'N/A')}")
        print(f"   Instructions count: {len(dest_customization.get('instructions', []))}")
        
        # Check text bases
        dest_text_response = get_text_bases(destination_content_base_uuid)
        dest_text_bases = dest_text_response.get("results", [])
        print(f"\n✅ Text bases: {len(dest_text_bases)} items")
        
        # Check link bases
        dest_link_bases = get_link_bases(destination_content_base_uuid)
        # Link bases return direct array
        print(f"✅ Link bases: {len(dest_link_bases)} items")
        
        # Check file bases
        dest_file_response = get_file_bases(destination_content_base_uuid)
        dest_file_bases = dest_file_response.get("results", [])
        print(f"✅ File bases: {len(dest_file_bases)} items")
        
        print("\n" + "=" * 50)
        print("Verification complete!")
        
    except Exception as e:
        print(f"❌ Error during verification: {str(e)}")

In [ ]:
# Run verification
verify_copy()